# Optimal Stock Assessments 

## Introduction + model description 
This note book shows how to used the KalmanFiltersPOMDP library to solve dynamic programming problems that include dcision to invest in informaiton. The model presnted here describes a resource mnamgnet agency investing in stock assesments to support a fishery. I assume the manager sets harvest limits on an anual basis with the goal of maximzing harvest  from the system: 

$max_{H_t}\left\{\sum_{t=0}^{\infty}\delta^tH_t \right\}$,

$s.t. H_t  > B_t$

$B_t = f_B(B_{t-1}, N_{t-1}) $,

$N_t = f_N(B_{t-1}, N_{t-1}) $

where $H_t$ is total harvest, $\delta$ is a discount factor. $B_t$ is the biomass of the popuatlion, $N_t$ describes the total number ofindividual in the population and the function $f_B$, $f_N$ describe the popautlion dynamics.

I assume that recruitment into the popualtion is a saturating function of the popualtion times a longnormal random varaible $\nu_t$ whcih described environmetnal variability

$R_t = \frac{rB_te^{\nu_t - 0.5\sigma_{nu_t}}}{1+bB_t}$.

I assume that the growth rate of each individuals is a declining function of size

$w_t = (1-k)w_{\infty} + kw_{t-1}$.

Taken together these two assumptions can be used to define the dynamics of a size structured popualtion that only tacks two state variables numbers and biomass.


I assume that the resource managers can observe the biomass of the popuatlion and the average size of the popualtion. I assume each of these observaiton is noisy and that the noise is normally distrubtied under a log transformation

$\hat{B}_t = B_te^{\epsilon_{B,t}}$

$\hat{w}_t = B_te^{\epsilon_{w,t}}/N_t$.

$\epsilon_{B,t} \sim N(0,\sigma_{b,t})$

$\epsilon_{w,t} \sim N(0,\sigma_{w,t})$

The resource manager can choose to invest in these observations each time period. If they do not invest in monitoring they recive a small amount of informaitn passively, i.e. $\sigma_{B,t}$ and $\sigma_{w,t}$ are large. If they do decide to monitor  I assume they recieve accurate informaiton about the popualtin stucture i.e. $\sigma_{w,t}$ is small, and some informaiton about the populations size. 


The manager makes decisions based on their beleif state which is defined by a subjective probability distribution over the state of the population. I assume that this distribution can be aproximated by a multivariate gausian density over the population biomass and numeric abundance. I use a technique called the Unscented Kalman Filter to compute how the managers belief state changes each time step. This algorithm aproxiates bayesian inferences integrating over the uncertianty in the biological porocess and incorperating inforamtion from the observations. 



## Define parameters for examples

I have two examples, on of a small productive popuatlion of a short lived speceis and one from a long lived 

In [184]:
include("BiomassDynamics.jl")

Main.BiomassDynamics

In [185]:
### short LH
# MSY + Fmsy
println("Short life history:")
msy,Fmsy = BiomassDynamics.Fmsy(BiomassDynamics.pars_short)
println("Fmsy = ", round(Fmsy,digits = 3))
println("MSY = ", round(msy,digits = 3))
# Bmsy
Nmsy,Bmsy = BiomassDynamics.equib(Fmsy, BiomassDynamics.pars_short)
println("Bmsy = ", round(Bmsy,digits = 3), "\n")

### long LH
println("Long life history:")
# MSY + Fmsy
msy,Fmsy = BiomassDynamics.Fmsy(BiomassDynamics.pars_long)
println("Fmsy = ", round(Fmsy,digits = 3))
println("MSY = ", round(msy,digits = 3))
# Bmsy
Nmsy,Bmsy = BiomassDynamics.equib(Fmsy, BiomassDynamics.pars_long)
println("Bmsy = ", round(Bmsy,digits = 3))

Short life history:
Fmsy = 0.1
MSY = 5.0
Bmsy = 52.586

Long life history:
Fmsy = 0.05
MSY = 5.0
Bmsy = 103.405


## Time series of belief dynamics and comparison to particle filter

In [209]:
state = BiomassDynamics.init(BiomassDynamics.pars_short)
HCR = x -> BiomassDynamics.p_star_sigma(x,0.1,0.2)
Sigma_O = [1.0 0.0; 0.0 0.1]
T = 100  
# Bt, Ht, Ft, Bhat, Sigma_Bt = BiomassDynamics.simulation(state,BiomassDynamics.pars_short,1,BiomassDynamics.Sigma_O_1,HCR,T)

100

In [208]:
include("BiomassDynamics.jl")
using Distributions
#### set parameters ####
# LH parameters
k = 0.75
winfty = 1.0
wr = 0.0
m = 0.25

# SR paramters
r = 2.00
b_ = 0.02
sigmaR = 0.5
dR = Distributions.Normal(0,sigmaR)

# pars list
pars = (k,winfty,wr,m,r,b_,sigmaR, 0.25)


### Observation parameters #####
sigma_obs_B = 0.25
sigma_obs_w = 0.05

omega = 1
Sigma_O = [sigma_obs_B 0.0;
           0.0 sigma_obs_w]


#### Harvest Control Rule ####
Ftarget = 0.05
pstar = 0.8
HCR = x -> BiomassDynamics.p_star_sigma(x,Ftarget,pstar)


#### Number of time steps ####
T = 500

#### Initialize State ####
N0 = 40.0
B0 = 20.0
xt = [log(N0),log(B0), 0.0]
xhat = [log(N0),log(B0), 0.0]
P0 = [1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 sigmaR]

state0 = BiomassDynamics.state(xt, 0.0,0.0, xhat, P0)



Bt, Ht, Ft, Sigma_Bt = BiomassDynamics.simulation(state0,pars,omega,Sigma_O,HCR,T)

LoadError: [91mUndefVarError: V not defined[39m

## Solution to observed system

### Value Function

### Policy Function

### Time series under optimal policy

## Solution to partially observed system 

### Value Function 

### Policy Function 

### Time series under opimal policy

### Mean and variance under alternative monitoring costs

### Frequency of monitoring for slow and fast life histories. 